# Fig6

In [1]:
from zebrafish_ms2_paper.gillespie_simulations_delay import Params, simulate_multiple_copies, sim_ms2, hill_function
from zebrafish_ms2_paper.trace_analysis import binarize_trace, get_on_and_off_times, get_burst_durations, get_burst_inactive_durations, extract_traces
from zebrafish_ms2_paper.utils import pboc_rc, style_axes, colors, fontsize
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import numpy as np
from multiprocessing import Pool
import pandas as pd
import pickle

In [107]:
rcParams.update(pboc_rc)
rcParams['pdf.fonttype'] = 42

In [3]:
colors

{'green': '#7AA974',
 'light_green': '#BFD598',
 'pale_green': '#DCECCB',
 'yellow': '#EAC264',
 'light_yellow': '#F3DAA9',
 'pale_yellow': '#FFEDCE',
 'blue': '#738FC1',
 'light_blue': '#A9BFE3',
 'pale_blue': '#C9D7EE',
 'red': '#D56C55',
 'light_red': '#E8B19D',
 'pale_red': '#F1D4C9',
 'purple': '#AB85AC',
 'light_purple': '#D4C2D9',
 'dark_green': '#7E9D90',
 'dark_brown': '#905426'}

In [73]:
fontsize = 9
linewidth = 2
run_sim = True
n_replicates = 15
markersize = 8
#bins = np.arange(3, 55)
bins = np.linspace(3, 55, 12)
#bins = np.linspace(3, 40, 15)


In [5]:
%matplotlib qt

In [6]:
"""specify the paths to the required datasets"""
path_to_dataset1 = r'/media/brandon/Data1/Somitogenesis/Dorado/manual_curation/complete_manual_spots.pkl'
path_to_non_blank_timepoints = r'/media/brandon/Data1/Somitogenesis/Dorado/non_blank_timepoints_full.pkl'

In [7]:
df = pd.read_pickle(path_to_dataset1)

In [106]:
len(df.nucleus_id.unique())

101

In [8]:
intens = df.gauss3d_dog.values
brightest_spot_intensity = np.quantile(intens[intens > 0], q=(0.95,))
brightest_spot_intensity / 10 / 0.29 / 4

array([4575.5862069])

In [9]:
"""estimate dynamic range of spot intensities to determine detection threshold for simluated data.
The intensities are approximately lognormally distributed. Let m, s be the mean and standard deviation respectively
of log10-transformed intensities. Then we estimate the detection threshold by

I_min = 10 ** (m - s)

In practice, we then take I_min 
"""
df = pd.read_pickle(path_to_dataset1)
intens = df.gauss3d_dog.values
plt.figure()
plt.hist(np.log10(intens[intens > 0]))
10**((np.mean(np.log10(intens[intens > 0])) - np.std(np.log10(intens[intens > 0]))) - (np.mean(np.log10(intens[intens > 0])) + np.std(np.log10(intens[intens > 0]))))


0.21940906682372294

In [10]:
m = np.mean(np.log10(intens[intens > 0]))
s = np.std(np.log10(intens[intens > 0]))

## Fig6B,C,D
Plot example traces for the three burst regulation models. Here we also plot the predicted protein traces, though we removed these from the final figure for simplicity. This code generates new stochastic simulations, so the traces won't be exactly the same was what is plotted in the figure.

In [110]:
def plot_example_trace(p, f=None, axs=None, burn_in_time=45):
    if f is None:
        f, axs = plt.subplots(2, 1, figsize=(4,3))

    X, tvec, p = simulate_multiple_copies(p)
    state = X[:, 0]
    mrna = X[:, -2]
    protein = X[:, -1]
    
    start_id = np.where(tvec > burn_in_time)[0][0]
    state = state[start_id:]
    mrna = mrna[start_id:]
    protein = protein[start_id:]
    tvec = tvec[start_id:] - burn_in_time
    X = X[start_id:]

    production_rate = p.transcription_rate_0 + p.transcription_rate_1 * hill_function(protein, p.KD_transcription_rate, p.n)
    #detection_threshold = 0.0
    relevant_production_rate = production_rate[np.where(tvec > burn_in_time)[0]]
    m = np.mean(np.log10(relevant_production_rate[relevant_production_rate > 0]))
    s = np.std(np.log10(relevant_production_rate[relevant_production_rate > 0]))
    detection_threshold = 0.22 * 10 ** (m + s) * w
    #detection_threshold = 0.2 * np.max(production_rate[np.where(tvec > burn_in_time)[0][0]]) * w

    ms2, t_obs = sim_ms2(state, tvec, production_rate, w, delta_t, sigma, detection_threshold)

    inferred_state = binarize_trace(ms2, t_obs, thresh=1e-1, window_size=3)

    # plot
    # state
    ax = axs[0]
    ax.plot(tvec, state, 'k-', linewidth=linewidth)

    #plt.title('amplitude regulation', fontsize=fontsize)
    ax.set_ylabel('promoter \nstate \n(0, 1)', fontsize=fontsize)
    ax.set_ylim([0,1.2])
    ax.set_xticks([0, 60, 120, 180], labels=[])
    ax = style_axes(ax, fontsize=fontsize)

    # MS2
    ax = axs[1]
    ax.plot(tvec, protein / np.max(protein), '-', linewidth=linewidth, color=colors['purple'], label='protein')
    ax.plot(t_obs, ms2 / np.max(ms2), '-', linewidth=linewidth, color=colors['green'], label='MS2')
    #ax.plot(t_obs, inferred_state, '-', linewidth=linewidth, color='k', label='inferred state')
    #ax.plot(tvec, production_rate / np.max(production_rate), '-', linewidth=linewidth, color=colors['blue'], label='transcription rate')
    
    #plt.title('amplitude regulation', fontsize=fontsize)
    ax.set_ylabel('simulated \nher1 activity \n(fraction of max)', fontsize=fontsize)
    ax.set_xlabel('time (min)', fontsize=fontsize)
    ax.set_ylim([0,1.5])
    ax.set_xticks([0, 60, 120, 180])
    ax.legend(fontsize=fontsize, facecolor='none', loc=1, ncol=2)
    ax = style_axes(ax, fontsize=fontsize)

    f.tight_layout()
    
    return f, axs, X, tvec

In [12]:
"""fixed model params"""
# w = 3.65
# delta_t = 1.0
# sigma = 0.2

elongation_time = 0.29
delta_t = 1.0
w = elongation_time / delta_t
sigma = 0.2

In [111]:
"""amplitude regulation, no bursts"""
p = Params()
p.initial_state = np.array([1])
p.Tmax = 240
p.k_off0 = 0#0.3#0.025
p.k_off1 = 0.0#0.0
p.k_on0 = 1000#0.1#.0
p.k_on1 = 0#2.5
p.transcription_rate_0 = 0

p.translation_rate = 4.5
p.transcription_rate_1 = 10.0 / len(p.initial_state)

p.mrna_decay_rate = 0.23
p.protein_decay_rate = 0.23

p.delay = 7.5

p.KD_transcription_rate = 100
p.n = 3

f, axs, X, tvec = plot_example_trace(p)

In [513]:
np.max(X[:,-1])

433.0

In [31]:
"""amplitude regulation with bursts"""
p = Params()
p.initial_state = np.array([1])
p.Tmax = 240
p.k_off0 = 0.4#0*0.08#025
p.k_off1 = 0.0#0.0
p.k_on0 = 4#0*0.4
p.k_on1 = 0#2.5
p.transcription_rate_0 = 0

p.translation_rate = 4.5
p.transcription_rate_1 = 10.0 / len(p.initial_state)

p.mrna_decay_rate = 0.23
p.protein_decay_rate = 0.23

p.delay = 7.5

p.KD_transcription_rate = 100
p.n = 3

f, axs, X, tvec = plot_example_trace(p)

In [81]:
"""k_on regulation"""
p = Params()
p.initial_state = np.array([1])#, 1, 1, 1])
p.Tmax = 240#1000
p.k_off0 = 0.2#0.2#2.2#0.025
p.k_off1 = 0.0#0.0
p.k_on0 = 0.0#0.1#2.2
p.k_on1 = 3#2.5
p.transcription_rate_0 = 10#10.0 / len(p.initial_state)

p.translation_rate = 4.5#4.5# max possible 16
p.transcription_rate_1 = 0 #10.0 / len(p.initial_state) max possible 50

p.mrna_decay_rate = 0.23
p.protein_decay_rate = 0.23

p.delay = 0.0

p.KD_k_on = 10
p.n = 10

f, axs, X, tvec = plot_example_trace(p)

In [561]:
"""k_off regulation"""
p = Params()
p.initial_state = np.array([1])#, 1, 1, 1])
p.Tmax = 240#1000
p.k_off0 = 0.0#0.2#2.2#0.025
p.k_off1 = 1000*0.08#0.0
p.k_on0 = 0.05#0.1#2.2
p.k_on1 = 0.0#2.5
p.transcription_rate_0 = 10#10.0 / len(p.initial_state)

p.translation_rate = 4.5#4.5# max possible 16
p.transcription_rate_1 = 0 #10.0 / len(p.initial_state) max possible 50

p.mrna_decay_rate = 0.23
p.protein_decay_rate = 0.23
p.delay = 0

p.KD_k_on = 10
p.KD_k_off = 800
p.n = 20
f, axs, X, tvec = plot_example_trace(p)

In [196]:
np.max(X[:,-1])

757.0

In [76]:
"""k_on and k_off regulation, complete regulation"""
p = Params()
p.number_of_random_numbers_to_pregenerate = 1e6
p.initial_state = np.array([1])#, 1, 1, 1])
p.Tmax = 240#1000
p.k_off0 = 0.0#0.2#0.2#2.2#0.025
p.k_off1 = 0.2#0.2#0.0
p.k_on0 = 0.0#0.1#0.1#2.2
p.k_on1 = 0.2#0.1#2.5
p.transcription_rate_0 = 10#10.0 / len(p.initial_state)

p.translation_rate = 4.5#4.5# max possible 16
p.transcription_rate_1 = 0 #10.0 / len(p.initial_state) max possible 50

p.mrna_decay_rate = 0.23
p.protein_decay_rate = 0.23
p.delay = 0

p.KD_k_on = 10
p.KD_k_off = 100
p.n = 10

f, axs, X, tvec = plot_example_trace(p)

In [54]:
"""k_on and k_off regulation, partial regulation"""
p = Params()
p.number_of_random_numbers_to_pregenerate = 1e6
p.initial_state = np.array([1])#, 1, 1, 1])
p.Tmax = 240#1000
p.k_off0 = 0.0#0.2#0.2#2.2#0.025
p.k_off1 = 0.2#0.2#0.0
p.k_on0 = 0.05#0.1#0.1#2.2
p.k_on1 = 0.1#0.1#2.5
p.transcription_rate_0 = 10#10.0 / len(p.initial_state)

p.translation_rate = 4.5#4.5# max possible 16
p.transcription_rate_1 = 0 #10.0 / len(p.initial_state) max possible 50

p.mrna_decay_rate = 0.23
p.protein_decay_rate = 0.23

p.delay = 0

p.KD_k_on = 1
p.KD_k_off = 100
p.n = 10

f, axs, X, tvec = plot_example_trace(p)

## Fig6E
For the three models of burst auto-regulation and one set of parameters each, compute the predicted interval distributions. We use multiprocessing for speed, but this can be circumvented by simply calling func below with a longer Tmax. 

In [41]:
"""fixed model params"""
# w = 3.65
# delta_t = 1.0
# sigma = 0.2
# Tmax = 1_000
#n_bins = 40
#bins = np.linspace(0, 60, n_bins)
n_replicates = 200
y_max = 0.15


In [18]:
"""function for computing interval distributions"""
def compute_burst_intervals(p):
    X, tvec, p = simulate_multiple_copies(p)
    burn_in_time = 0
    X = X[burn_in_time:]
    production_rate = p.transcription_rate_0 + p.transcription_rate_1 * hill_function(X[:,-1], p.KD_transcription_rate, p.n)
    state = X[:, 0]

    elongation_time = 0.29
    delta_t = 1.0
    w = elongation_time / delta_t
    sigma = 0.2
    relevant_production_rate = production_rate[np.where(tvec > burn_in_time)[0]]
    m = np.mean(np.log10(relevant_production_rate[relevant_production_rate > 0]))
    s = np.std(np.log10(relevant_production_rate[relevant_production_rate > 0]))
    detection_threshold = 0.22 * 10 ** (m + s) * w

    #detection_threshold = 0
    ms2, uniform_times = sim_ms2(state, tvec, production_rate, w, delta_t, sigma, detection_threshold)
    
    inferred_state = binarize_trace(ms2, uniform_times, thresh=1e-1, window_size=3)
    on_times, off_times = get_on_and_off_times(inferred_state, uniform_times)
    active_durations  = get_burst_durations(on_times, off_times)
    inactive_durations = get_burst_inactive_durations(on_times, off_times)
    periods = np.diff(on_times)
    
    """tmp plotting"""
#     protein = X[:, -1]
#     f, ax = plt.subplots()
#     ax.plot(tvec, protein / np.max(protein), '-', linewidth=linewidth, color=colors['purple'], label='protein')
#     ax.plot(uniform_times, ms2 / np.max(ms2), '-', linewidth=linewidth, color=colors['green'], label='MS2')
#     ax.plot(uniform_times, inferred_state, '-', linewidth=linewidth, color='k', label='inferred state')
    
#     #plt.title('amplitude regulation', fontsize=fontsize)
#     ax.set_ylabel('simulated \nher1 activity \n(fraction of max)', fontsize=fontsize)
#     ax.set_xlabel('time (min)', fontsize=fontsize)
#     ax.set_ylim([0,1.5])
#     ax.set_xticks([0, 60, 120, 180])
#     ax.legend(fontsize=fontsize, facecolor='none', loc=1, ncol=2)
#     ax = style_axes(ax, fontsize=fontsize)

#     f.tight_layout()


    
    
    return active_durations, inactive_durations, periods

def init_pool_processes():
    np.random.seed()
    

"""more functions for computing interval distributions. we also need func and init_pool_processes from the cell above."""
def compute_distributions(traces, bins):
    pulse_periods = []
    pulse_durations =[]
    pulse_quiets = []
    for i, trace in enumerate(traces):
        t_arr, inten_arr, nucleus = trace
        t_arr = non_blank_timepoints[t_arr.astype('int')]
        state = binarize_trace(inten_arr, t_arr, thresh=1.0, window_size=5)
        on_times, off_times = get_on_and_off_times(state, t_arr)
        if len(on_times) > 2:
            these_pulse_periods = np.diff(on_times)
            these_quiets = get_burst_inactive_durations(on_times, off_times)
            pulse_periods.extend([p for p in these_pulse_periods])
            these_pulse_durations = get_burst_durations(on_times, off_times)
            pulse_durations.extend([p for p in these_pulse_durations])
            pulse_quiets.extend([p for p in these_quiets])
            
    counts, bins = np.histogram(pulse_durations, bins=bins)
    prob_dens_durations = counts / np.sum(counts) / np.diff(bins)
    
    counts, bins = np.histogram(pulse_quiets, bins=bins)
    prob_dens_quiets = counts / np.sum(counts) / np.diff(bins)
            
    return prob_dens_durations, prob_dens_quiets


def bootstrap_distributions(traces, bins, n_bootstraps):
    duration_dist_arr = np.zeros((n_bootstraps, len(bins) - 1))
    quiet_dist_arr = np.zeros((n_bootstraps, len(bins) - 1))
    for i in range(int(n_bootstraps)):
        these_ids = np.random.randint(0, len(traces), len(traces), dtype='int')
        these_traces = [traces[j] for j in these_ids]
        
        duration_dist_arr[i], quiet_dist_arr[i] = compute_distributions(these_traces, bins)

    return duration_dist_arr, quiet_dist_arr 


def bootstrap_simulated_distributions(intervals, bins, n_bootstraps):
    interval_dist_arr = np.zeros((n_bootstraps, len(bins) - 1))
    for i in range(int(n_bootstraps)):
        these_ids = np.random.randint(0, len(intervals), len(intervals), dtype='int')
        these_intervals = [intervals[j] for j in these_ids]
        
        counts, bins = np.histogram(these_intervals, bins=bins)
        interval_dist_arr[i] = counts / np.sum(counts) / np.diff(bins)
        
    return interval_dist_arr 

## Run sims

In [42]:
from copy import deepcopy

In [43]:
"""amplitude regulation, with some bursts"""
save = True
if run_sim:
    p = Params()
    p.initial_state = np.array([1])
    p.Tmax = 240
    p.k_off0 = 0.4#0*0.08#0#0.3#0.025
    p.k_off1 = 0.0#0.0
    p.k_on0 = 4#0*0.4#1000#0.1#.0
    p.k_on1 = 0#2.5
    p.transcription_rate_0 = 0

    p.translation_rate = 4.5
    p.transcription_rate_1 = 10.0 / len(p.initial_state)

    p.mrna_decay_rate = 0.23
    p.protein_decay_rate = 0.23

    p.delay = 7.5

    p.KD_transcription_rate = 100#25#75
    p.n = 3#2.2#3
    p_arr = [p] * n_replicates #* 4
    decay_scale = 0*0.23 / 3
    for i in range(len(p_arr)):
        tmp_p = deepcopy(p_arr[i])
        tmp_p.protein_decay_rate = np.clip(np.random.normal(loc=0.23, scale=decay_scale), a_min=0, a_max=np.inf)
        p_arr[i] = tmp_p

    with Pool(processes=15, initializer=init_pool_processes) as pool:
        res = pool.map(compute_burst_intervals, p_arr)

    active_durations = [item for sublist in res for item in sublist[0]]
    inactive_durations = [item for sublist in res for item in sublist[1]]
    periods = [item for sublist in res for item in sublist[2]]

if save:
    with open(r'/home/brandon/Documents/Code/zebrafish-ms2-paper/data/delay_sims/amp_reg_bursts_intervals.pkl', 'wb') as f:
        pickle.dump([active_durations, inactive_durations, periods], f)






In [44]:
"""amplitude regulation, with no bursts"""
save = True
if run_sim:
    p = Params()
    p.initial_state = np.array([1])
    p.Tmax = 240
    p.k_off0 = 0.0
    p.k_off1 = 0.0
    p.k_on0 = 0.0
    p.k_on1 = 0.0
    p.transcription_rate_0 = 0

    p.translation_rate = 4.5
    p.transcription_rate_1 = 10.0 / len(p.initial_state)

    p.mrna_decay_rate = 0.23
    p.protein_decay_rate = 0.23

    p.delay = 7.5

    p.KD_transcription_rate = 100#25#75
    p.n = 3
    p_arr = [p] * n_replicates * 4
    decay_scale = 0.23 / 3
    for i in range(len(p_arr)):
        tmp_p = deepcopy(p_arr[i])
        tmp_p.protein_decay_rate = np.clip(np.random.normal(loc=0.23, scale=decay_scale), a_min=0, a_max=np.inf)
        p_arr[i] = tmp_p

    with Pool(processes=15, initializer=init_pool_processes) as pool:
        res = pool.map(compute_burst_intervals, p_arr)

    active_durations = [item for sublist in res for item in sublist[0]]
    inactive_durations = [item for sublist in res for item in sublist[1]]
    periods = [item for sublist in res for item in sublist[2]]

if save:
    with open(r'/home/brandon/Documents/Code/zebrafish-ms2-paper/data/delay_sims/amp_reg_intervals.pkl', 'wb') as f:
        pickle.dump([active_durations, inactive_durations, periods], f)






In [45]:
"""frequency regulation"""
save = True
if run_sim:
    p = Params()
    p.initial_state = np.array([1])#, 1, 1, 1])
    p.Tmax = 240#1000
    p.k_off0 = 0.08#0.2#2.2#0.025
    p.k_off1 = 0.0#0.0
    p.k_on0 = 0.0#0.1#2.2
    p.k_on1 = 0.5#2.5
    p.transcription_rate_0 = 10#10.0 / len(p.initial_state)

    p.translation_rate = 4.5#4.5# max possible 16
    p.transcription_rate_1 = 0 #10.0 / len(p.initial_state) max possible 50

    p.mrna_decay_rate = 0.23
    p.protein_decay_rate = 0.23

    p.delay = 0.0

    p.KD_k_on = 80#10
    p.n = 3

    p_arr = [p] * n_replicates
    decay_scale = 0*0.23 / 3
    for i in range(len(p_arr)):
        tmp_p = deepcopy(p_arr[i])
        tmp_p.protein_decay_rate = np.clip(np.random.normal(loc=0.23, scale=decay_scale), a_min=0, a_max=np.inf)
        p_arr[i] = tmp_p
        
    with Pool(processes=15, initializer=init_pool_processes) as pool:
        res = pool.map(compute_burst_intervals, p_arr)

    active_durations = [item for sublist in res for item in sublist[0]]
    inactive_durations = [item for sublist in res for item in sublist[1]]
    periods = [item for sublist in res for item in sublist[2]]

if save:
    with open(r'/home/brandon/Documents/Code/zebrafish-ms2-paper/data/delay_sims/freq_reg_intervals.pkl', 'wb') as f:
        pickle.dump([active_durations, inactive_durations, periods], f)



In [46]:
"""duration regulation"""
save = True
if run_sim:
    p = Params()
    p.initial_state = np.array([1])#, 1, 1, 1])
    p.Tmax = 240#1000
    p.k_off0 = 0.0#0.2#2.2#0.025
    p.k_off1 = 0.4#0.0
    p.k_on0 = 0.055#0.1#2.2
    p.k_on1 = 0.0#2.5
    p.transcription_rate_0 = 10#10.0 / len(p.initial_state)

    p.translation_rate = 4.5#4.5# max possible 16
    p.transcription_rate_1 = 0 #10.0 / len(p.initial_state) max possible 50

    p.mrna_decay_rate = 0.23
    p.protein_decay_rate = 0.23
    p.delay = 0

    p.KD_k_on = 10
    p.KD_k_off = 1100#1500#1100#800
    p.n = 3

    p_arr = [p] * n_replicates
    decay_scale = 0*0.23 / 3
    for i in range(len(p_arr)):
        tmp_p = deepcopy(p_arr[i])
        tmp_p.protein_decay_rate = np.clip(np.random.normal(loc=0.23, scale=decay_scale), a_min=0, a_max=np.inf)
        p_arr[i] = tmp_p
        
    with Pool(processes=15, initializer=init_pool_processes) as pool:
        res = pool.map(compute_burst_intervals, p_arr)

    active_durations = [item for sublist in res for item in sublist[0]]
    inactive_durations = [item for sublist in res for item in sublist[1]]
    periods = [item for sublist in res for item in sublist[2]]

if save:
    with open(r'/home/brandon/Documents/Code/zebrafish-ms2-paper/data/delay_sims/dur_reg_intervals.pkl', 'wb') as f:
        pickle.dump([active_durations, inactive_durations, periods], f)



In [47]:
"""freq + duration regulation"""
save = True
if run_sim:
    p = Params()
    p.number_of_random_numbers_to_pregenerate = 1e6
    p.initial_state = np.array([1])#, 1, 1, 1])
    p.Tmax = 240#1000
    p.k_off0 = 0.0#0.2#0.2#2.2#0.025
    p.k_off1 = 0.4#0.2#0.0
    p.k_on0 = 0.0#0.1#0.1#2.2
    p.k_on1 = 0.5#0.1#2.5
    p.transcription_rate_0 = 10#10.0 / len(p.initial_state)

    p.translation_rate = 4.5#4.5# max possible 16
    p.transcription_rate_1 = 0 #10.0 / len(p.initial_state) max possible 50

    p.mrna_decay_rate = 0.23
    p.protein_decay_rate = 0.23
    p.delay = 0

    p.KD_k_on = 80
    p.KD_k_off = 1300#1500
    p.n = 3
    p_arr = [p] * n_replicates
    decay_scale = 0*0.23 / 3
    for i in range(len(p_arr)):
        tmp_p = deepcopy(p_arr[i])
        tmp_p.protein_decay_rate = np.clip(np.random.normal(loc=0.23, scale=decay_scale), a_min=0, a_max=np.inf)
        p_arr[i] = tmp_p
        
    with Pool(processes=15, initializer=init_pool_processes) as pool:
        res = pool.map(compute_burst_intervals, p_arr)

    active_durations = [item for sublist in res for item in sublist[0]]
    inactive_durations = [item for sublist in res for item in sublist[1]]
    periods = [item for sublist in res for item in sublist[2]]

if save:
    with open(r'/home/brandon/Documents/Code/zebrafish-ms2-paper/data/delay_sims/freq_and_dur_reg_intervals.pkl', 'wb') as f:
        pickle.dump([active_durations, inactive_durations, periods], f)



In [100]:
def plot_interval_dists(intervals, 
                        bins,
                        ax=None,
                        xlabel=None, 
                        ylabel=None, 
                        xticks=(0, 15, 30), 
                        yticks=(0, 0.05, 0.10),
                        n_bootstraps=100,
                        title=None,
                        color='k',
                       ):
    if ax is None:
        f, ax = plt.subplots()
    counts, bins = np.histogram(intervals, bins=bins)
    prob_dens = counts / np.sum(counts) / np.diff(bins)
    interval_dist_arr = bootstrap_simulated_distributions(intervals, bins, n_bootstraps=n_bootstraps)
    prob_dens_uncertainty = np.std(interval_dist_arr, axis=0)
    #ax.plot(bins[:-1], prob_dens, 'ko', markersize=markersize, markerfacecolor=colors['green'], alpha=0.75, label='data')
#     ax.errorbar(bins[:-1], prob_dens, prob_dens_uncertainty, 
#                 color=colors['green'], linewidth=linewidth, 
#                 elinewidth=linewidth, ecolor=colors['green'], 
#                 capsize=linewidth, capthick=linewidth,
#                 marker='o', markersize=markersize, 
#                 markerfacecolor='none',
#                 markeredgecolor=colors['green'],
#                    markeredgewidth=linewidth)
    ax.fill_between(bins[:-1], prob_dens - prob_dens_uncertainty, prob_dens + prob_dens_uncertainty,
                   facecolor=color, alpha=0.5)
    ax.plot(bins[:-1], prob_dens, '-', linewidth=linewidth, color=color)
    
    if xlabel is None:
        xtick_labels = []
    else:
        xtick_labels = xticks
    if ylabel is None:
        ytick_labels = []
    else:
        ytick_labels = yticks
    ax.set_xticks(xticks, labels=xtick_labels)
    ax.set_xlabel(xlabel, fontsize=fontsize)
    ax.set_yticks(yticks, labels=ytick_labels)
    ax.set_ylabel(ylabel, fontsize=fontsize)
    if title is not None:
        ax.set_title(title, fontsize=fontsize, fontweight='bold')
        
    ax = style_axes(ax, fontsize=fontsize)
    
    return ax, prob_dens

def load_intervals(file_name):
    with open(file_name, 'rb') as f:
        active_intervals, inactive_intervals, periods = pickle.load(f)
    
    return active_intervals, inactive_intervals, periods
        
def plot_simulated_dists(axd, keys, sim_dir, files_to_load, counter=0, titles=None, xlabel=None, ylabel=None, colors=None):
    for i in range(len(files_to_load)):
        if titles is not None:
            title = titles[i]
        else:
            title= None
                   
        active_intervals, inactive_intervals, _ = load_intervals(sim_dir + '/' + files_to_load[i])
        ax = axd[keys[counter]]
        if colors is not None:
            color = colors[counter]
        else:
            color = 'k'
        ax, prob_dens = plot_interval_dists(active_intervals, bins, ax, xlabel=xlabel, ylabel=ylabel, title=title, color=color)
        ax.set_ylim([0, 0.12])
        ax.set_xlim([0, 40])

        counter += 1
        
        ax = axd[keys[counter]]
        if colors is not None:
            color = colors[counter]
        else:
            color = 'k'
        ax, prob_dens = plot_interval_dists(inactive_intervals, bins, ax, xlabel=xlabel, ylabel=ylabel, title=title, color=color)
        ax.set_xlim([0, 40])
        ax.set_ylim([0, 0.12])

        counter += 1
    
    return counter

In [49]:
#plt.close('all')

In [103]:
"""plot grouped by active vs inactive interval"""
f, axd = plt.subplot_mosaic([['a', 'b', 'c', 'd', 'e'], 
                             ['f', 'g', 'h', 'i', 'j']], figsize=(6.5, 4))
keys = ['a', 'f', 'b', 'g', 'c', 'h','d', 'i', 'e', 'j']
titles = ['amplitude \nregulation', 'frequency \nregulation', 'duration \nregulation', 'frequency + duration \nregulation', 'data']
sim_dir = r'/home/brandon/Documents/Code/zebrafish-ms2-paper/data/delay_sims'
files_to_load = ['amp_reg_intervals.pkl', 'freq_reg_intervals.pkl', 'dur_reg_intervals.pkl', 'freq_and_dur_reg_intervals.pkl']
sim_colors = (colors['blue'], colors['purple'],) * 5
counter = 0
xticks = [0, 30, 60]

counter = plot_simulated_dists(axd, keys, sim_dir, files_to_load, counter=0, xlabel='time (min)', titles=titles, colors=sim_colors)
with open(r'/home/brandon/Documents/Code/zebrafish-ms2-paper/data/manual_data_intervals/manual_data_intervals.pkl', 'rb') as f:
    active_intervals, inactive_intervals, periods = pickle.load(f)

data_bins = np.linspace(0, 35, 10)
ax = axd[keys[counter]]
ax, prob_dens = plot_interval_dists(active_intervals, data_bins, ax, color=colors['green'], xlabel='time (min)')
ax.set_xlabel('time (min)', fontsize=fontsize)
ax.set_xlim([0, 40])
ax.set_ylim([0, 0.12])
ax.set_title('experimental \ndata', fontsize=fontsize)
counter += 1

ax = axd[keys[counter]]
ax, prob_dens = plot_interval_dists(inactive_intervals, data_bins, ax, color=colors['green'], xlabel='time (min)')
ax.set_xlabel('time (min)', fontsize=fontsize)
ax.set_xlim([0, 40])
ax.set_ylim([0, 0.12])
ax.set_title('experimental \ndata', fontsize=fontsize)

axd['a'].set_ylabel('probability \ndensity (1/min)', fontsize=fontsize)
axd['a'].set_yticks((0, 0.05, 0.10), labels=(0, 0.05, 0.10))

axd['f'].set_ylabel('probability \ndensity (1/min)', fontsize=fontsize)
axd['f'].set_yticks((0, 0.05, 0.10), labels=(0, 0.05, 0.10))

plt.gcf().tight_layout(w_pad=-0.15)

In [104]:
plt.savefig(r'/media/brandon/Data1/Somitogenesis/Dorado/interval_dists_delay_sims_colors.pdf')

## Fit mean and std. dev of interval distributions

In [51]:
def compute_loss_function(p, n_replicates=15, mode='both'):
    p_arr = [p] * n_replicates

    with Pool(processes=15, initializer=init_pool_processes) as pool:
        res = pool.map(compute_burst_intervals, p_arr)

    active_durations = [item for sublist in res for item in sublist[0]]
    inactive_durations = [item for sublist in res for item in sublist[1]]
    
    active_loss = ((np.mean(active_durations) - data_mean_active) ** 2 
                + (np.std(active_durations) - data_std_active) ** 2
                  )
    
    inactive_loss = ((np.mean(inactive_durations) - data_mean_inactive) ** 2
                    + (np.std(inactive_durations) - data_std_inactive) ** 2
                    )
#     """try just fitting ratios"""
#     active_loss = ((np.mean(active_durations) / np.std(active_durations) - data_mean_active / data_std_active) ** 2)
    
#     inactive_loss = ((np.mean(inactive_durations) / np.std(inactive_durations) - data_mean_inactive / data_std_inactive) ** 2)
    
    if mode == 'both':
        loss = active_loss + inactive_loss
    elif mode == 'active':
        loss = active_loss
    elif mode == 'inactive':
        loss = inactive_loss
    else:
        raise ValueError
    
    return loss

In [52]:
with open(r'/home/brandon/Documents/Code/zebrafish-ms2-paper/data/manual_data_intervals/manual_data_intervals.pkl', 'rb') as f:
    active_intervals, inactive_intervals, periods = pickle.load(f)

data_mean_active = np.mean(active_intervals)
data_std_active = np.std(active_intervals)
data_mean_inactive = np.mean(inactive_intervals)
data_std_inactive = np.std(inactive_intervals)


In [53]:
data_mean_active

12.71830985915493

In [54]:
data_mean_active / data_std_active

2.2209200924025367

In [55]:
# """fit amplitude"""
# hill_coeff_arr = np.linspace(1, 10, 6)
# KD_arr = np.logspace(0, np.log10(300), 5)
# loss_arr_amp = np.zeros((len(hill_coeff_arr), len(KD_arr)))

# p = Params()
# p.initial_state = np.array([1])
# p.Tmax = 240
# p.k_off0 = 0#0.3#0.025
# p.k_off1 = 0.0#0.0
# p.k_on0 = 1000#0.1#.0
# p.k_on1 = 0#2.5
# p.transcription_rate_0 = 0

# p.translation_rate = 4.5
# p.transcription_rate_1 = 10.0 / len(p.initial_state)

# p.mrna_decay_rate = 0.23
# p.protein_decay_rate = 0.23

# p.delay = 6.0

# for i in range(len(hill_coeff_arr)):
#     print(f'{i+1} of {len(hill_coeff_arr)}')
#     for j in range(len(KD_arr)):
#         p.KD = KD_arr[j]
#         p.n = hill_coeff_arr[i]
#         loss_arr_amp[i, j] = compute_loss_function(p)

In [56]:
#loss_arr_amp = loss_arr

In [57]:
# plt.figure()
# plt.contourf(np.log10(KD_arr), hill_coeff_arr, loss_arr_amp)
# plt.xlabel('log10(KD)', fontsize=fontsize)
# plt.ylabel('hill coefficient', fontsize=fontsize)
# plt.colorbar()

In [58]:
# """fit duration"""
# hill_coeff_arr = np.linspace(2, 24, 6)
# KD_arr = np.logspace(0, np.log10(1000), 5)
# loss_arr_dur = np.zeros((len(hill_coeff_arr), len(KD_arr)))

# p = Params()
# p.initial_state = np.array([1])#, 1, 1, 1])
# p.Tmax = 240#1000
# p.k_off0 = 0.0#0.2#2.2#0.025
# p.k_off1 = 0.08#0.0
# p.k_on0 = 0.05#0.1#2.2
# p.k_on1 = 0.0#2.5
# p.transcription_rate_0 = 10#10.0 / len(p.initial_state)

# p.translation_rate = 4.5#4.5# max possible 16
# p.transcription_rate_1 = 0 #10.0 / len(p.initial_state) max possible 50

# p.mrna_decay_rate = 0.23
# p.protein_decay_rate = 0.23
# p.delay = 0


# for i in range(len(hill_coeff_arr)):
#     print(f'{i+1} of {len(hill_coeff_arr)}')
#     for j in range(len(KD_arr)):
#         p.KD = KD_arr[j]
#         p.n = hill_coeff_arr[i]
#         loss_arr_dur[i, j] = compute_loss_function(p, n_replicates=15, mode='active')

In [59]:
# plt.figure()
# plt.contourf(np.log10(KD_arr), hill_coeff_arr, loss_arr_dur)
# plt.xlabel('log10(KD)', fontsize=fontsize)
# plt.ylabel('hill coefficient', fontsize=fontsize)
# plt.colorbar()

In [60]:
# np.where(loss_arr_dur == np.min(loss_arr_dur))
# print(f'n={hill_coeff_arr[np.where(loss_arr_dur == np.min(loss_arr_dur))[0][0]]}')
# print(f'KD={KD_arr[np.where(loss_arr_dur == np.min(loss_arr_dur))[1][0]]}')

## plot example burst schematic

In [81]:
"""amplitude regulation, no bursts"""
p = Params()
p.initial_state = np.array([1])
p.Tmax = 120
p.k_off0 = 0.1#0.3#0.025
p.k_off1 = 0.0#0.0
p.k_on0 = 0.05#0.1#.0
p.k_on1 = 0#2.5
p.transcription_rate_0 = 0

p.translation_rate = 4.5
p.transcription_rate_1 = 10.0 / len(p.initial_state)

p.mrna_decay_rate = 0.23
p.protein_decay_rate = 0.23

p.delay = 6.0

p.KD_transcription_rate = 400
p.n = 4

f, axs, X, tvec = plot_example_trace(p)

In [95]:
t_arr = np.linspace(0, 100)
state = np.zeros_like(t_arr)
state[np.array(t_arr > 25) * np.array(t_arr < 50)] = 1
state[np.array(t_arr > 75) * np.array(t_arr < 90)] = 1
plt.figure(figsize=(2.5, 1.5))
plt.plot(t_arr, state, 'k-', linewidth=linewidth)
plt.xlabel('time (min)', fontsize=fontsize)
plt.ylabel('promoter \nstate', fontsize=fontsize)
plt.yticks([0, 1], labels=[])
ax = style_axes(plt.gca(), fontsize=fontsize)
plt.tight_layout()

In [96]:
plt.savefig(r'/media/brandon/Data1/Somitogenesis/Dorado/burst_schematic.pdf')